# Eat Safe, Love

## Notebook Set Up

In [35]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd


In [36]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [37]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [38]:
# review the collections in our database
db.list_collection_names()


['establishments']

In [39]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [40]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene" : 20}
results = establishments.find(query)
# Use count_documents to display the number of documents in the result
print(f" The result has this many documents: {establishments.count_documents(query)}\n")
# Display the first document in the results using pprint
pprint(establishments.find_one(query))

 The result has this many documents: 0

None


In [41]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f"The dataframe has this many rows: {len(result_df)}")
# Display the first 10 rows of the DataFrame
result_df.head(10)

The dataframe has this many rows: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [42]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query2 = {'LocalAuthorityName': {'$regex': 'London'}, 
          'RatingValue': {'$gte': '4'}
         }

# Use count_documents to display the number of documents in the result
count2 = establishments.count_documents(query2)
print(f"There are {count2} this many establishments in London that have a RatingValue greater than or equal to 4.")

# Display the first document in the results using pprint
pprint(establishments.find_one(query2))

There are 0 this many establishments in London that have a RatingValue greater than or equal to 4.
None


In [43]:
# Convert the result to a Pandas DataFrame
df2 = pd.DataFrame(establishments.find(query2))

# Display the number of rows in the DataFrame
print(f"The Dataframe has this many rows: {len(df2)}.")

# Display the first 10 rows of the DataFrame
result_df.head(10)

The Dataframe has this many rows: 0.


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [44]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
restaurants = establishments.find_one({"BusinessName": "Penang Flavours"}, {"geocode": 1,})
degree_search = 0.01
latitude = float(str(restaurants['geocode']['latitude']))
longitude = float(str(restaurants['geocode']['longitude']))

query = {'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
        'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
        'RatingValue': 5}
sort = [('scores.Hygiene', 1)]
limit = 5

result = list(establishments.find(query).sort(sort).limit(limit))
# Print the results
pprint(result)

[]


In [45]:
# Convert result to Pandas DataFrame
restaurants_hygiene_df = pd.DataFrame(result)
restaurants_hygiene_df.head()


""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [46]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
match_query = {"$match": {"scores.Hygiene" : 0}}
# 2. Groups the matches by Local Authority
group_query = {"$group": {"_id": "$LocalAuthorityName",
                          "count": {"$sum":1}}}
# 3. Sorts the matches from highest to lowest
sort_values = {'$sort': {'count': -1}}
# Put the pipeline together
pipeline = [match_query, group_query, sort_values]
# Run aggregation line and save the results to a variable
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print('The document numbers in the result are:', len(results))
# Print the first 10 results
pprint(results[:10])

The document numbers in the result are: 0
[]


In [47]:
# Convert the result to a Pandas DataFrame
zero_hygiene_df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print('The dataframe has this many rows:', len(zero_hygiene_df))
# Display the first 10 rows of the DataFrame
zero_hygiene_df.head(10)


The dataframe has this many rows: 0


""
